In [42]:
%pip install brokenaxes==0.2
%pip install fpdf
%pip install openai
# %pip install numpy==1.26.4
# %pip install pandas==2.2.3
# %pip install sklearn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import seaborn as sns
import geopandas as gpd

import re
from brokenaxes import brokenaxes
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from scipy.stats import mstats

import os
import zipfile
import shutil
from fpdf import FPDF
from openai import OpenAI
import base64

In [45]:
merged = pd.read_csv('merged.csv')
calculated = pd.read_csv('calculated.csv')

We are going to build a pipeline that use OpenAI to automatically generate insights from graphs.

In [56]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [117]:
client = OpenAI(
    api_key=("YOURKEY")
)
def summarize(prt):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system",
             "content": "You are a data scientist that can generate business or financial insights from graphs. Use pure text instead of markdown. You will be provided with the binary string of the graph, you need to carefully read the data and graphs and text on it, provide a financial report for the management leadership. You will carefully read graph, grab the data and information, and provide the insights that can be gained from the graph, and provide strategies recommendations accordingly."},
            {"role": "user", "content": [
                {
                    "type": "text",
                    "text": "What financial or company insights can be drawn from this image?",
                }, 
                {
                  "type": "image_url",
                  "image_url": {
                    "url":  f"data:image/jpeg;base64,{prt}"
                  }
                }
            ]
            }
        ]
    )
    return completion.choices[0].message.content

In [118]:
# Warning: cost money. This cell only needs to run once
# Reorder the png
selected_images = [
    'Profit Efficiency by Project.png',
    'Profit Efficiency by Department.png',
    'Profit Efficiency by Department and Project.png',
    'Project Classification.png',
    'Number of Projects by 谈判状态.png',
    'Average Profit by 谈判状态.png',
    'Profit Efficiency by Project for Large Projects.png',
    'Profit Efficiency by Project for Medium Projects.png',
    'Profit Efficiency by Project for Small Projects.png',
    'Profit Efficiency by Department for Large Projects.png',
    'Profit Efficiency by Department for Medium Projects.png',
    'Profit Efficiency by Department for Small Projects.png',
    'Profit Efficiency by Department and Project for Large.png',
    'Profit Efficiency by Department and Project for Medium.png',
    'Profit Efficiency by Department and Project for Small.png',
    'Boxplot of Hours by Department.png',
    'Labor Efficiency by All Department.png',
    'Labor Efficiency by Department - Below Benchmark (100).png'
]

# Encode and call openAI
print('processing images...')
encoded_images = [encode_image(x) for x in selected_images]
print('analyzing images...will take a while...')
responses = [summarize(x) for x in encoded_images]

processing images...
analyzing images...will take a while...


In [138]:
good="1234567890-=_+qwertyuiop[]{}\\|asdfghjkl;':\"zxcvbnm,./<>?~`\t\n\rQWERTYUIOPLKJHGFDSAZXCVBNM !@#$%^&*()"
columns = list(merged.columns)
for col in columns:
    chinese = merged[col].unique()
    for cha in chinese:
        good = good + str(cha)
response2 = []
for x in responses:
    response2.append('')
    for y in x:
        if y in good:
            response2[-1] += y

In [155]:
# Create a PDF object
pdf = FPDF()
pdf.add_font('NotoSans', '', 'NotoSansCJKsc-Regular.ttf', uni=True)
pdf.set_auto_page_break(auto=True, margin=15)
image_dir = os.getcwd()
# Check if report.pdf already exists and delete it
output_pdf_path = 'final_report.pdf'
if os.path.exists(output_pdf_path):
    os.remove(output_pdf_path)
    print(f'{output_pdf_path} removed.')

# Add each selected image to the PDF
for image_file, text in zip(selected_images, response2):
    print('current: ', image_file)
    image_path = os.path.join(image_dir, image_file)
    if os.path.exists(image_path):  # Check if the file exists
        pdf.add_page()
        print("adding page")
        pdf.image(image_path, x=10, y=10, w=pdf.w - 20)
        pdf.ln(110)  # Space after the image (adjust based on image height)
        print('adjusting layout...')
        # Add the text below the image
        pdf.set_font('NotoSans', '', 12)
        print(text)
        pdf.multi_cell(0, 10, text)  # `multi_cell` wraps text across lines if it's too long
        print('writing insights...')
    
# Save the PDF to a file
output_pdf_path = 'final_report.pdf'
pdf.output(output_pdf_path)
print(f"PDF saved to {output_pdf_path}")

print('--------Finished running---------')

final_report.pdf removed.
current:  Profit Efficiency by Project.png
adding page
adjusting layout...
The graph illustrates the median profit efficiency (profit per work hour) for various projects, highlighting the top five and bottom five performers.

Insights:

1. **Top Performer:**
   - The project at the top of the graph, labeled 北京大学兰园书院设计项目, shows significantly higher profit efficiency, far surpassing all other projects. This indicates it is exceptionally profitable per work hour.

2. **Bottom Performers:**
   - The bottom project, 深圳龙岗金荣达6楼MAG华为专属场地前台综合升级改造项目, shows a significant negative profit efficiency. This suggests it is not only unprofitable but costing the company significantly in terms of resources per hour.
   - Several other projects also show negative efficiency but to a lesser extent.

Recommendations:

1. **Focus on High Performers:**
   - Investigate the factors contributing to the success of the top project to replicate its efficiency in future projects.

2. **Add

/srv/conda/envs/notebook/lib/python3.11/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33445
  warnings.warn("cmap value too big/small: %s" % cm)
/srv/conda/envs/notebook/lib/python3.11/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33638
  warnings.warn("cmap value too big/small: %s" % cm)
/srv/conda/envs/notebook/lib/python3.11/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -34233
  warnings.warn("cmap value too big/small: %s" % cm)
/srv/conda/envs/notebook/lib/python3.11/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -34508
  warnings.warn("cmap value too big/small: %s" % cm)
/srv/conda/envs/notebook/lib/python3.11/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -34698
  warnings.warn("cmap value too big/small: %s" % cm)
/srv/conda/envs/notebook/lib/python3.11/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -35004
  warning

PDF saved to final_report.pdf
--------Finished running---------


After running, download final report from [here](https://data101.datahub.berkeley.edu/user/shuran1030/files/final_report.pdf?_xsrf=MnwxOjB8MTA6MTcyOTkwMTA3NXw1Ol94c3JmfDEzMjpNR05sTVRVMk5UUXlaVEE1TkRJNFl6a3pOekk1WkdFNFlXRXpNV1UyTjJRNllqQTVaV1l3WWpnNVkyWXpOelV4WTJReVlqaGtabU16WWpWak4yTTBZMkUzWkRRM09HRXlNVEZrTXpFNFl6RTNOVGxrTXpreU5HRm1OelV5WWpNNFlRPT18OTEwOTliZjYwMWFkYWJiNzZjMjM4ZDM2NWZjZDQ1NTAxM2RhZThhOGNmMDI3YWNkY2Y4MmQxODQ0ZjZjMTkwMA)